In [2]:
from pydub import AudioSegment, silence
from pydub.effects import normalize

import numpy as np
from pathlib import Path
from pydub.playback import play
import noisereduce as nr

/home/lex/.local/share/virtualenvs/GenerativeMusic-JrvY4iMK/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
source_dir = '/home/lex/Data/Guzheng/Guzheng/categorised by material (traditional school of guzheng playing)'
school_dir = '_Alienated_ materials'
fname = 'alienated 1 靖沐dress rehearsal.wav'

In [26]:
path = f'{source_dir}/{school_dir}/{fname}'
clip = AudioSegment.from_file(path)
clip = clip.set_channels(1)

In [43]:
noise = AudioSegment.from_file(f'{source_dir}/{school_dir}/alienated 1 靖沐第一次錄音.wav')
noise = normalize(noise)[9900:]
noise_raw = np.array(noise.get_array_of_samples())
noise_raw = noise_raw
with Path('./noise_from_alienated 1 靖沐第一次錄音.wav').open('wb') as f: 
    noise.export(f, format='wav')
play(noise)

Input #0, wav, from '/tmp/tmpzw1wn71f.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:03.00, bitrate: 1411 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, 2 channels, s16, 1411 kb/s


In [38]:
clip_norm = normalize(clip)

clip_raw = np.array(clip_norm.get_array_of_samples())

clip_raw_no_noise = nr.reduce_noise(y=clip_raw, 
                                    sr=clip_norm.frame_rate, 
                                    y_noise=noise_raw, 
                                    stationary=True, 
                                    prop_decrease=0.8
                                    )

clip_no_noise = AudioSegment(clip_raw_no_noise.tobytes(), 
                             frame_rate=clip.frame_rate, 
                             channels=1, 
                             sample_width=clip_raw_no_noise.dtype.itemsize)

non_silent = silence.split_on_silence(clip_no_noise, silence_thresh=-40, keep_silence=500)
clip_no_silence = sum(non_silent[1:], start=non_silent[0]) if len(non_silent) > 1 else non_silent[0]

In [39]:
len(clip_norm), len(clip_no_noise),len(clip_no_silence)

(13000, 13000, 11132)

In [40]:
play(clip_norm[:5000])

Input #0, wav, from '/tmp/tmpci11yyvk.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:05.00, bitrate: 705 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, 1 channels, s16, 705 kb/s


In [41]:
play(clip_no_noise[:5000])

Input #0, wav, from '/tmp/tmpwsqtrmvh.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:05.00, bitrate: 705 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, 1 channels, s16, 705 kb/s


In [42]:
play(clip_no_silence)

Input #0, wav, from '/tmp/tmpu5lc3en0.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:11.13, bitrate: 705 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, 1 channels, s16, 705 kb/s
